In [293]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [294]:
df = pd.read_csv('cardiovascular_clean.csv', delimiter=',')
df.head()

,sbp,ldl,adiposity,typea,alcohol,age,chd,tabaco_anual,famhist_Absent,famhist_Present,alcohol_over_median,tabaco_over_median,age_over_median,ldl_over_median,adiposity_over_median,sbp_over_median
0,160,5.73,23.11,49,97.20,52,1,0.324324,False,True,True,True,True,True,False,True
1,144,4.41,28.61,55,2.06,63,1,0.000208,True,False,False,False,True,True,True,True
2,118,3.48,32.28,52,3.81,46,0,0.002581,False,True,False,False,True,False,True,False
3,170,6.41,38.03,51,24.26,58,1,0.174419,False,True,True,True,True,True,True,True
4,134,3.50,27.78,60,57.34,49,1,0.400000,False,True,True,True,True,False,True,False


### Modelos para edad mayor de la media

In [295]:
dfage=df[df['age_over_median']==True]
dfage.head()

,sbp,ldl,adiposity,typea,alcohol,age,chd,tabaco_anual,famhist_Absent,famhist_Present,alcohol_over_median,tabaco_over_median,age_over_median,ldl_over_median,adiposity_over_median,sbp_over_median
0,160,5.73,23.11,49,97.20,52,1,0.324324,False,True,True,True,True,True,False,True
1,144,4.41,28.61,55,2.06,63,1,0.000208,True,False,False,False,True,True,True,True
2,118,3.48,32.28,52,3.81,46,0,0.002581,False,True,False,False,True,False,True,False
3,170,6.41,38.03,51,24.26,58,1,0.174419,False,True,True,True,True,True,True,True
4,134,3.50,27.78,60,57.34,49,1,0.400000,False,True,True,True,True,False,True,False


In [296]:
X = dfage.drop(['chd','alcohol_over_median','tabaco_over_median','age_over_median','ldl_over_median','adiposity_over_median','sbp_over_median'], axis=1)
y = dfage['chd']

In [297]:
X.head()

,sbp,ldl,adiposity,typea,alcohol,age,tabaco_anual,famhist_Absent,famhist_Present
0,160,5.73,23.11,49,97.20,52,0.324324,False,True
1,144,4.41,28.61,55,2.06,63,0.000208,True,False
2,118,3.48,32.28,52,3.81,46,0.002581,False,True
3,170,6.41,38.03,51,24.26,58,0.174419,False,True
4,134,3.50,27.78,60,57.34,49,0.400000,False,True


In [298]:
#Normalizamos con standarscaler - escala entre 0 y variacion estandars
#scaler = StandardScaler()
scaler =MinMaxScaler()
X=scaler.fit_transform(X)
X

array([[4.72727273e-01, 3.31010453e-01, 3.57427056e-01, ...,
        4.57380457e-01, 0.00000000e+00, 1.00000000e+00],
       [3.27272727e-01, 2.39024390e-01, 5.39787798e-01, ...,
        2.93803419e-04, 1.00000000e+00, 0.00000000e+00],
       [9.09090909e-02, 1.74216028e-01, 6.61472149e-01, ...,
        3.63937138e-03, 0.00000000e+00, 1.00000000e+00],
       ...,
       [6.72727273e-01, 2.39024390e-01, 6.55503979e-01, ...,
        1.60083160e-01, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 4.25087108e-02, 9.61538462e-02, ...,
        1.05769231e-01, 1.00000000e+00, 0.00000000e+00],
       [2.18181818e-01, 2.67595819e-01, 6.98938992e-01, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [299]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=270724)
X_train.shape

(154, 9)

In [300]:
X_test.shape

(67, 9)

### Reg Logistica

Sin SMOTE

In [301]:
# Crear el modelo de regresión logística
model = LogisticRegression(solver='lbfgs', max_iter=1000)
# Validación cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=270724)
cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
# Entrenar el modelo
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [302]:
# Evaluar el modelo
y_pred = model.predict(X_test)

# Calcular la exactitud
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Reporte de clasificación
class_report = classification_report(y_test, y_pred)
print('Classification Report age sin SMOTE:')
print(class_report)

# Resultados de validación cruzada
print(f'Cross-validation accuracy scores: {cv_scores}')
print(f'Mean cross-validation accuracy: {cv_scores.mean():.2f}')

Accuracy: 0.60
Confusion Matrix:
[[21 15]
 [12 19]]
Classification Report age sin SMOTE:
              precision    recall  f1-score   support

           0       0.64      0.58      0.61        36
           1       0.56      0.61      0.58        31

    accuracy                           0.60        67
   macro avg       0.60      0.60      0.60        67
weighted avg       0.60      0.60      0.60        67

Cross-validation accuracy scores: [0.75       0.5        0.625      0.75       0.6        0.66666667
 0.53333333 0.73333333 0.46666667 0.73333333]
Mean cross-validation accuracy: 0.64


Con SMOTE y best model

In [303]:
smote = SMOTE(random_state=270724)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=270724)
# Realizar modelo regresión logística
model = LogisticRegression(solver = "lbfgs", max_iter = 1000) # Le aplicamos el modelo lbfgs en particular + cuantas veces queremos entrenar el modelo con los datos
cv = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 270724) # Pedimos que mezcle datos y semilla
cv_score = cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy') # con métrica accuracy

# Entrenamos modelo
model.fit(X_train, y_train)
# Predicción de y
y_pred = model.predict(X_test)

# Métricas
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy = {accuracy:.2f}")
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)
reporte = classification_report(y_test, y_pred)
print(f"Classification Report age con SMOTE: {reporte}")


Accuracy = 0.60
Confusion Matrix:
[[21 15]
 [12 19]]
Classification Report age con SMOTE:               precision    recall  f1-score   support

           0       0.64      0.58      0.61        36
           1       0.56      0.61      0.58        31

    accuracy                           0.60        67
   macro avg       0.60      0.60      0.60        67
weighted avg       0.60      0.60      0.60        67



In [304]:
X_train.shape

(154, 9)

In [305]:
X_train_res.shape

(158, 9)

In [306]:
# Data aumentation para simular mas muestra

No hay diferencia si lo hacemos con SMOTE porque solo crea 4 registros,
Coger el más optimo tampoco influye

#SVM

In [307]:
svm_rbf = SVC ( kernel='sigmoid', random_state=270224)
param_grid_rbf = {
    'C': [0.1, 1, 10, 25, 50, 100],
    'gamma': [1, 0.1, 0.01, 0.001]
}
grid_search_rbf = GridSearchCV(svm_rbf,
                               param_grid_rbf,
                               cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=270224),
                               scoring = 'accuracy',
                               verbose = 1,
                               n_jobs=-1)
grid_search_rbf.fit(X_train_res,y_train_res)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=270224, shuffle=True),
             estimator=SVC(kernel='sigmoid', random_state=270224), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 25, 50, 100],
                         'gamma': [1, 0.1, 0.01, 0.001]},
             scoring='accuracy', verbose=1)

In [308]:
best_rbf = grid_search_rbf.best_estimator_
best_rbf

SVC(C=50, gamma=0.1, kernel='sigmoid', random_state=270224)

In [309]:
y_pred=best_rbf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'SVM Regression Accuracy: {accuracy:.2f}')

conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix age - SVM:')
print(conf_matrix)

class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)

SVM Regression Accuracy: 0.64
Confusion Matrix age - SVM:
[[23 13]
 [11 20]]
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.64      0.66        36
           1       0.61      0.65      0.62        31

    accuracy                           0.64        67
   macro avg       0.64      0.64      0.64        67
weighted avg       0.64      0.64      0.64        67



##Modelos para ldl mayor de la media

In [310]:
dfldl=df[df['ldl_over_median']==True]
dfldl.head()

,sbp,ldl,adiposity,typea,alcohol,age,chd,tabaco_anual,famhist_Absent,famhist_Present,alcohol_over_median,tabaco_over_median,age_over_median,ldl_over_median,adiposity_over_median,sbp_over_median
0,160,5.73,23.11,49,97.20,52,1,0.324324,False,True,True,True,True,True,False,True
1,144,4.41,28.61,55,2.06,63,1,0.000208,True,False,False,False,True,True,True,True
3,170,6.41,38.03,51,24.26,58,1,0.174419,False,True,True,True,True,True,True,True
5,132,6.47,36.21,62,14.14,45,0,0.206667,False,True,True,True,False,True,True,False
7,114,4.59,14.60,62,6.72,58,1,0.094884,False,True,False,True,True,True,False,False


In [311]:
dfldl.count()

sbp                      230
ldl                      230
adiposity                230
typea                    230
alcohol                  230
age                      230
chd                      230
tabaco_anual             230
famhist_Absent           230
famhist_Present          230
alcohol_over_median      230
tabaco_over_median       230
age_over_median          230
ldl_over_median          230
adiposity_over_median    230
sbp_over_median          230
dtype: int64

In [312]:
X = dfldl.drop(['chd','alcohol_over_median','tabaco_over_median','age_over_median','ldl_over_median','adiposity_over_median','sbp_over_median'], axis=1)
y = dfldl['chd']

In [313]:
#Normalizamos con standarscaler - escala entre 0 y variacion estandars
#scaler = StandardScaler()
scaler = MinMaxScaler()
X=scaler.fit_transform(X)
X

array([[5.22123894e-01, 1.24087591e-01, 4.14501511e-01, ...,
        4.79867623e-01, 0.00000000e+00, 1.00000000e+00],
       [3.80530973e-01, 3.64963504e-03, 5.80664653e-01, ...,
        3.08248299e-04, 1.00000000e+00, 0.00000000e+00],
       [6.10619469e-01, 1.86131387e-01, 8.65256798e-01, ...,
        2.58068344e-01, 0.00000000e+00, 1.00000000e+00],
       ...,
       [7.16814159e-01, 3.64963504e-03, 6.86102719e-01, ...,
        1.67953668e-01, 1.00000000e+00, 0.00000000e+00],
       [1.50442478e-01, 6.60583942e-01, 6.46525680e-01, ...,
        3.19591837e-01, 1.00000000e+00, 0.00000000e+00],
       [2.74336283e-01, 4.10583942e-02, 7.25679758e-01, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

# Reg Logistica

In [314]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=270724)
X_train.shape

(161, 9)

sin SMOTE

In [315]:
# Crear el modelo de regresión logística
model = LogisticRegression(solver='lbfgs', max_iter=1000)
# Validación cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=270724)
cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
# Entrenar el modelo
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [316]:
# Evaluar el modelo
y_pred = model.predict(X_test)

# Calcular la exactitud
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Reporte de clasificación
class_report = classification_report(y_test, y_pred)
print('Classification Report Report ldl sin SMOTE:')
print(class_report)

# Resultados de validación cruzada
print(f'Cross-validation accuracy scores: {cv_scores}')
print(f'Mean cross-validation accuracy: {cv_scores.mean():.2f}')

Accuracy: 0.70
Confusion Matrix:
[[23  8]
 [13 25]]
Classification Report Report ldl sin SMOTE:
              precision    recall  f1-score   support

           0       0.64      0.74      0.69        31
           1       0.76      0.66      0.70        38

    accuracy                           0.70        69
   macro avg       0.70      0.70      0.70        69
weighted avg       0.70      0.70      0.70        69

Cross-validation accuracy scores: [0.64705882 0.6875     0.4375     0.625      0.6875     0.75
 0.6875     0.75       0.5625     0.8125    ]
Mean cross-validation accuracy: 0.66


Con SMOTE y best Model

In [317]:
smote = SMOTE(random_state=270724)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=270724)
# Realizar modelo regresión logística
model = LogisticRegression(solver = "lbfgs", max_iter = 1000) # Le aplicamos el modelo lbfgs en particular + cuantas veces queremos entrenar el modelo con los datos
cv = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 270724) # Pedimos que mezcle datos y semilla
cv_score = cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy') # con métrica accuracy

# Entrenamos modelo
model.fit(X_train, y_train)
# Predicción de y
y_pred = model.predict(X_test)

# Métricas
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy = {accuracy:.2f}")
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)
reporte = classification_report(y_test, y_pred)
print(f"Classification Report ldl con SMOTE : {reporte}")


Accuracy = 0.70
Confusion Matrix:
[[23  8]
 [13 25]]
Classification Report ldl con SMOTE :               precision    recall  f1-score   support

           0       0.64      0.74      0.69        31
           1       0.76      0.66      0.70        38

    accuracy                           0.70        69
   macro avg       0.70      0.70      0.70        69
weighted avg       0.70      0.70      0.70        69



In [318]:
''' Con StandardScaler
Accuracy = 0.67
Confusion Matrix:
[[22  9]
 [14 24]]
Classification Report ldl con SMOTE :               precision    recall  f1-score   support

           0       0.61      0.71      0.66        31
           1       0.73      0.63      0.68        38

    accuracy                           0.67        69
   macro avg       0.67      0.67      0.67        69
weighted avg       0.68      0.67      0.67        69
'''

' Con StandardScaler\nAccuracy = 0.67\nConfusion Matrix:\n[[22  9]\n [14 24]]\nClassification Report ldl con SMOTE :               precision    recall  f1-score   support\n\n           0       0.61      0.71      0.66        31\n           1       0.73      0.63      0.68        38\n\n    accuracy                           0.67        69\n   macro avg       0.67      0.67      0.67        69\nweighted avg       0.68      0.67      0.67        69\n'

No hay diferencia si lo hacemos con SMOTE porque solo crea 4 registros,
Coger el más optimo tampoco influye.

In [319]:
#Matriz con edad con StandardScaler
#[[22 14]
 #[10 21]]


Con MaxScaler mejoramos accuracy y matriz de confusion

#SVM

In [320]:
svm_rbf = SVC ( kernel='sigmoid', random_state=270224)
param_grid_rbf = {
    'C': [0.1, 1, 10, 25, 50, 100],
    'gamma': [1, 0.1, 0.01, 0.001]
}
grid_search_rbf = GridSearchCV(svm_rbf,
                               param_grid_rbf,
                               cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=270224),
                               scoring = 'accuracy',
                               verbose = 1,
                               n_jobs=-1)
grid_search_rbf.fit(X_train_res,y_train_res)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=270224, shuffle=True),
             estimator=SVC(kernel='sigmoid', random_state=270224), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 25, 50, 100],
                         'gamma': [1, 0.1, 0.01, 0.001]},
             scoring='accuracy', verbose=1)

In [321]:
best_rbf = grid_search_rbf.best_estimator_
best_rbf

SVC(C=50, gamma=0.1, kernel='sigmoid', random_state=270224)

In [322]:
y_pred=best_rbf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'SVM Regression Accuracy: {accuracy:.2f}')

conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix age - LDL:')
print(conf_matrix)

class_report = classification_report(y_test, y_pred)
print('Classification Report age - LDL:')
print(class_report)

SVM Regression Accuracy: 0.64
Confusion Matrix age - LDL:
[[16 15]
 [10 28]]
Classification Report age - LDL:
              precision    recall  f1-score   support

           0       0.62      0.52      0.56        31
           1       0.65      0.74      0.69        38

    accuracy                           0.64        69
   macro avg       0.63      0.63      0.63        69
weighted avg       0.64      0.64      0.63        69



Funciona mejor R.Logista que SVM

In [323]:
#Coger solo las 3 con mas correlacion

#Modelos 3 var mas correlacionadas

In [324]:
df3c=df[['age','ldl','adiposity','chd']]
df3c.head()

,age,ldl,adiposity,chd
0,52,5.73,23.11,1
1,63,4.41,28.61,1
2,46,3.48,32.28,0
3,58,6.41,38.03,1
4,49,3.50,27.78,1


In [325]:
X = df3c.drop(['chd'], axis=1)
y = df3c['chd']

In [326]:
#Normalizamos con standarscaler - escala entre 0 y variacion estandars
#scaler = StandardScaler()
scaler = MinMaxScaler()
X=scaler.fit_transform(X)
X

array([[0.75510204, 0.33101045, 0.4579021 ],
       [0.97959184, 0.23902439, 0.61174825],
       [0.63265306, 0.17421603, 0.71440559],
       ...,
       [0.81632653, 0.04250871, 0.23748252],
       [0.51020408, 0.74076655, 0.67272727],
       [0.63265306, 0.26759582, 0.74601399]])

In [327]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=270724)

In [328]:
# Crear el modelo de regresión logística
model = LogisticRegression(solver='lbfgs', max_iter=1000)
# Validación cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=270724)
cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
# Entrenar el modelo
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [329]:
# Evaluar el modelo
y_pred = model.predict(X_test)

# Calcular la exactitud
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Reporte de clasificación
class_report = classification_report(y_test, y_pred)
print('Classification Report Report 3 var sin SMOTE:')
print(class_report)

# Resultados de validación cruzada
print(f'Cross-validation accuracy scores: {cv_scores}')
print(f'Mean cross-validation accuracy: {cv_scores.mean():.2f}')

Accuracy: 0.65
Confusion Matrix:
[[73  8]
 [41 17]]
Classification Report Report 3 var sin SMOTE:
              precision    recall  f1-score   support

           0       0.64      0.90      0.75        81
           1       0.68      0.29      0.41        58

    accuracy                           0.65       139
   macro avg       0.66      0.60      0.58       139
weighted avg       0.66      0.65      0.61       139

Cross-validation accuracy scores: [0.66666667 0.66666667 0.78787879 0.6875     0.6875     0.75
 0.65625    0.75       0.625      0.75      ]
Mean cross-validation accuracy: 0.70


Al quitar variables el resultado empeora mucho en la deteccion de 'tiene enfermedad' (1)

con SMOTE y best model

In [330]:
smote = SMOTE(random_state=270724)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=270724)
# Realizar modelo regresión logística
model = LogisticRegression(solver = "lbfgs", max_iter = 1000) # Le aplicamos el modelo lbfgs en particular + cuantas veces queremos entrenar el modelo con los datos
cv = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 270724) # Pedimos que mezcle datos y semilla
cv_score = cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy') # con métrica accuracy

# Entrenamos modelo
model.fit(X_train, y_train)
# Predicción de y
y_pred = model.predict(X_test)

# Métricas
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy = {accuracy:.2f}")
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)
reporte = classification_report(y_test, y_pred)
print(f"Classification Report 3 var con SMOTE :\n {reporte}")

Accuracy = 0.65
Confusion Matrix:
[[73  8]
 [41 17]]
Classification Report 3 var con SMOTE :
               precision    recall  f1-score   support

           0       0.64      0.90      0.75        81
           1       0.68      0.29      0.41        58

    accuracy                           0.65       139
   macro avg       0.66      0.60      0.58       139
weighted avg       0.66      0.65      0.61       139



Modelos 4 var mas correlacionadas

In [331]:
df4c=df[['age','ldl','adiposity','tabaco_anual','chd']]
df4c.head()

,age,ldl,adiposity,tabaco_anual,chd
0,52,5.73,23.11,0.324324,1
1,63,4.41,28.61,0.000208,1
2,46,3.48,32.28,0.002581,0
3,58,6.41,38.03,0.174419,1
4,49,3.50,27.78,0.400000,1


In [332]:
X = df4c.drop(['chd'], axis=1)
y = df4c['chd']

In [333]:
#Normalizamos con standarscaler - escala entre 0 y variacion estandars
#scaler = StandardScaler()
scaler = MinMaxScaler()
X=scaler.fit_transform(X)
X

array([[7.55102041e-01, 3.31010453e-01, 4.57902098e-01, 4.57380457e-01],
       [9.79591837e-01, 2.39024390e-01, 6.11748252e-01, 2.93803419e-04],
       [6.32653061e-01, 1.74216028e-01, 7.14405594e-01, 3.63937138e-03],
       ...,
       [8.16326531e-01, 4.25087108e-02, 2.37482517e-01, 1.05769231e-01],
       [5.10204082e-01, 7.40766551e-01, 6.72727273e-01, 3.04615385e-01],
       [6.32653061e-01, 2.67595819e-01, 7.46013986e-01, 0.00000000e+00]])

In [334]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=270724)

In [335]:
# Crear el modelo de regresión logística
model = LogisticRegression(solver='lbfgs', max_iter=1000)
# Validación cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=270724)
cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
# Entrenar el modelo
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [336]:
# Evaluar el modelo
y_pred = model.predict(X_test)

# Calcular la exactitud
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Reporte de clasificación
class_report = classification_report(y_test, y_pred)
print('Classification Report Report 4 var sin SMOTE:')
print(class_report)

# Resultados de validación cruzada
print(f'Cross-validation accuracy scores: {cv_scores}')
print(f'Mean cross-validation accuracy: {cv_scores.mean():.2f}')

Accuracy: 0.68
Confusion Matrix:
[[73  8]
 [37 21]]
Classification Report Report 4 var sin SMOTE:
              precision    recall  f1-score   support

           0       0.66      0.90      0.76        81
           1       0.72      0.36      0.48        58

    accuracy                           0.68       139
   macro avg       0.69      0.63      0.62       139
weighted avg       0.69      0.68      0.65       139

Cross-validation accuracy scores: [0.6969697  0.66666667 0.78787879 0.71875    0.6875     0.75
 0.59375    0.75       0.65625    0.75      ]
Mean cross-validation accuracy: 0.71


con smote y best model

In [337]:
smote = SMOTE(random_state=270724)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=270724)
# Realizar modelo regresión logística
model = LogisticRegression(solver = "lbfgs", max_iter = 1000) # Le aplicamos el modelo lbfgs en particular + cuantas veces queremos entrenar el modelo con los datos
cv = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 270724) # Pedimos que mezcle datos y semilla
cv_score = cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy') # con métrica accuracy

# Entrenamos modelo
model.fit(X_train, y_train)
# Predicción de y
y_pred = model.predict(X_test)

# Métricas
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy = {accuracy:.2f}")
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)
reporte = classification_report(y_test, y_pred)
print(f"Classification Report 4 var con SMOTE :\n {reporte}")

Accuracy = 0.68
Confusion Matrix:
[[73  8]
 [37 21]]
Classification Report 4 var con SMOTE :
               precision    recall  f1-score   support

           0       0.66      0.90      0.76        81
           1       0.72      0.36      0.48        58

    accuracy                           0.68       139
   macro avg       0.69      0.63      0.62       139
weighted avg       0.69      0.68      0.65       139

